# December 2017: Advent of Code Solutions
## Daniel Näslund
From Dec. 1 to Dec. 25, [I](dannas.name) will be solving the puzzles that appear each day at [Advent of Code](http://adventofcode.com/). The two-part puzzles are released at midnight EST (6:00AM CET); points are awarded to the first 100 people to solve the day's puzzles. 

To understand the problems completely, you will have to read the full description in the "[Day 1](http://adventofcode.com/2017/day/1):" link in each day's section header.

##  Prelude
Here I import common functions and modules so I don't have to do it each day.
These are borrowed from [Peter Norvigs Advent of Code solutions](https://github.com/norvig/pytudes/blob/master/ipynb/Advent%20of%20Code.ipynb) from 2016. I've also reused his ipython notebook layout.

In [1]:
# Python 3.x
from itertools import islice, cycle, permutations, count as count_from
from collections import Counter, defaultdict, deque
import re

cat = ''.join
def Input(day):
    filename = 'advent2017/input{}.txt'.format(day);
    return open(filename);
    # TODO(dannas): Fetch the files from elsewhere to allow remote access

def InputStr(day):
    filename = 'advent2017/input{}.txt'.format(day)
    return open(filename).read().rstrip('\n')

def quantify(iterable, pred=bool):
    return sum(map(pred, iterable))
    
# 2-D points implemented using (x, y) tuples
def X(point): return point[0]
def Y(point): return point[1]

def move(pos, direction):
    return (X(pos) + X(direction), Y(pos) + Y(direction))

def neighbors4(point):
    "The four neighbors (without diagnoals)"
    x, y = point
    return ((x+1, y), (x-1, y), (x, y+1), (x, y-1))

def neighbors8(point):
    "The eight neighbors (with diagonals)"
    x, y =  point
    return ((x+1, y), (x-1, y), (x, y+1), (x, y-1),
           (x+1, y+1), (x-1, y-1), (x+1, y-1), (x-1, y+1))

def cityblock_distance(p, q=(0, 0)): 
    "City block distance between two points."
    return abs(X(p) - X(q)) + abs(Y(p) - Y(q))


def first(iterable):
    return next(iter(iterable))

def nth(iterable, n, default=None):
    "Returns the nth item or a default value"
    return next(islice(iterable, n, None), default)

def head(iterable, n=10):
    return iterable[:10]

# TODO(dannas): Maybe accept list input as well?
def array(string):
    return [vector(line) for line in string.splitlines()]

def vector(line):
    return [atom(word) for word in line.split()]

# TODO(dannas): Check floats as well
def atom(word):
    try:
        return int(word)
    except ValueError:
        return word

def mapt(fn, iterable):
    return tuple(map(fn, iterable))

def ints(line):
    matches = re.findall(r'\d+', line)
    return mapt(int, matches)

def grep(pattern, file):
    for line in file:
        if re.search(pattern, line):
            print(line)
        

## [Day 1](http://adventofcode.com/2017/day/1): Inverse Captcha
Given a file of digits, find the sum of all digits that match the next digit in the list. The list is circular, so the digit after the last digit is the first digit in the list.

We need to parse with one token lookahead, it's enough to append the first digit to the end of the list for handling the circular case.

In [ ]:
def pairs(seq):
    return zip(seq[:-1], seq[1:])

def solve_captcha(str):
    digits = [int(d) for d in str] 
    if len(digits) < 2:
        return 0
    digits.append(digits[0])
    return sum(x for x, y in pairs(digits) if x == y)
    
solve_captcha(Input(1).read().strip())

In **part 2** we shall compare the digit halfway around the circular list to the current one. The list has an even number of elements.

This require N/2 token lookahead instad of one. I could have appended the first half of the list to the end, but instead I opted for a circular queue. A circular queue can be implicitely represented using indexes that wrap around; or explicitely using an [ADT](https://docs.python.org/3.6/library/collections.html?highlight=deque#collections.deque.rotate); or using operations on iterators. I choose the later.

The [cycle()](https://docs.python.org/3.6/library/itertools.html#itertools.cycle) function provides an iterator to an infinite circular representation of the list. With [islice](https://docs.python.org/3.6/library/itertools.html#itertools.islice) I can select my start and end position in that list.

In [ ]:
def pairs(seq):
    N = len(seq)
    half = int(N/2)
    x = islice(cycle(seq), 0, N)
    y = islice(cycle(seq), half, N + half)
    return zip(x, y)

def solve_captcha(str):
    digits = [int(d) for d in str] 
    return sum(x for x, y in pairs(digits) if x == y)
    
solve_captcha(Input(1).read().strip())


## [Day 2](http://adventofcode.com/2017/day/2): Corruption Checksum
For each row in a spreadsheet, determine the difference between largest and smallest value; the checksum is the sum of all of these differences.

I initially had some trouble parsing the input into a list of list of integers. It's been a while since I used Python.

In [ ]:
def maxmins(spreadsheet):
    for row in spreadsheet:
        yield max(row), min(row)

def parse(line):
    return tuple(int(x) for x in line.split())

spreadsheet = [parse(line) for line in Input(2)]
sum(x-y for x,y in maxmins(spreadsheet))

In **part 2** we're asked to find the only two numbers in each row where one evenly divides the other - that is, where the result of the division operation is a whole number. Find those numbers on each line, divide them, and add up each line's result.

In [ ]:
def evens(spreadsheet):
   for row in spreadsheet:
        for x, y in permutations(row, 2):
            if x % y == 0:
                yield x, y
                
def parse(line):
    return tuple(int(x) for x in line.split())

spreadsheet = [parse(line) for line in Input(2)]
sum(x/y for x,y in evens(spreadsheet))


## [Day 3](http://adventofcode.com/2017/day/3): Spiral Memory
Walk a a grid in a spiral pattern, a specified number of steps. Then calculate the manhattan distance to the origin.

The trace of steps will form a nested set of cubes. I tried to come up with a neat formula for describing how the the number of points in those cubes increases. Failed. Instead I experimented on paper and found that we increase the sides when we turn East and West.

I decided to represent the points using x,y tuples. I considered using complex numbers first. The walk() function was originally a long list of explicit steps. I then extracted the common parts and ended up with the move and turn functions.

Representing the direction as vectors (2 element tuples of x and y) and using vector addition was a heureka moment for me.

In [5]:
E, N, W, S = [(1, 0), (0, 1), (-1, 0), (0, -1)]    

def move(pos, direction):
    return (X(pos) + X(direction), Y(pos) + Y(direction))

def turn(direction, num_steps):
    changes = {
        E : (N, 0),
        N : (W, 1),
        W : (S, 0),
        S : (E, 1),
    }
    d, delta = changes[direction]
    return d, num_steps + delta


def walk(total_steps):
    pos = (0, 0)
    num_steps = 1
    walked = 0
    direction = E
    
    while total_steps > 0:
        walked += 1
        pos = move(pos, direction)
        if walked == num_steps:
            direction, num_steps = turn(direction, num_steps)
            walked = 0

        total_steps -= 1
    return pos

pos = walk(289326 - 1)

cityblock_distance(pos)

419

In **part 2** we're asked to, for each field, store the sum of previously visited fields, including diagonals. The center point has value 1. Once a field is written, its value doesn't change. 

I spent a lot of time trying to come up with a clever algorithm for determining which neighbors to a field was already visited. Then I had an euphyfany and realized that I could compare all 8 possible adjacent fields, if I had stored a mapping between visited fields and their sums. Determine the first value written that is larger than the puzzle input.

In [ ]:
square_values = {}

def neighbour_sum(pos, direction):
    return sum(square_values[N] for N in neighbors8(pos) if N in square_values)
   
def walk(lower_limit):
    pos = (0, 0)
    square_values[pos] = 1
    num_steps = 1
    walked = 0
    direction = E
    
    while square_values[pos] <= lower_limit:
        walked += 1
        pos = move(pos, direction)
        square_values[pos] = neighbour_sum(pos, direction)
        if walked == num_steps:
            direction, num_steps = turn(direction, num_steps)
            walked = 0
    return square_values[pos]

walk(289326)

## [Day 4](http://adventofcode.com/2017/day/4): High-Entropy Passphrases
How many pass phrases are valid, a.k.a. do not contain duplicated words?

In [ ]:
lines = [L for L in Input(4)]

def is_valid(line):
    words = line.split()
    return len(words) == len(set(words))

sum(1 if is_valid(line) else 0 
    for line in lines)

In **part 2** we're asked how many passphrases are valid, this time with the added requirement that no words in the passphrase can be an anagram of another word.

I remember this one from the chapter Aha! Algorithms in the book Programming Pearls. In that book, Jon meant that detecting anagrams is an example of an algorithm that people may bank their head against the wall with until they finally hits the insight. Thankfully, I was already familiar with it.

In [ ]:
def is_valid(line):
    words = [cat(sorted(w)) for w in line.split()]
    return len(words) == len(set(words))

sum(1 if is_valid(line) else 0
     for line in lines)

## [Day 5](http://adventofcode.com/2017/day/5): A Maze of Twisty Trampolines, All Alike
Given a list of relative jumps, determine the number of steps neccessary to reach a destination outside the list. There's a quirk: The jump instructions increment by one upon being visited.

I first wrote this as a straightforward loop. Had one bug: I interpreted the jumps as absolute positions at first. I'm borrowing the quantify function which I found in Peter Norvigs 2017 Advent of Code solution Jupyter notebook.

In [ ]:
def jumps(instr):
    pos = 0
    while pos >= 0 and pos < N:
        old = pos
        yield pos
        pos += instr[pos]
        instr[old] += 1
    yield pos
        
quantify(jumps([int(x) for x in Input(5)]))

In **part 2** we're asked to again calculate number of steps. After each jump, if the offset was 3 or more, decrement it by 1 else increment it by 1.

I wrote jumps2 as a function that returned the number of steps. Then I rewrote it to yield values instead. That caused the running time to increase from "almost instant" to 8s. Doh. Keeping it as-is, since I accidentily removed the original solution (and I don't understand how  Jupyter handles editing history - I want my vi editor instead of  this webbrowser madness!)

In [ ]:
def jumps2(instr):
    pos = 0
    while pos >= 0 and pos < N:
        old = pos
        yield pos
        offset = instr[pos]
        pos += instr[pos]
        if offset >= 3:
            instr[old] -= 1
        else:
            instr[old] += 1
    yield pos

quantify(jumps2([int(x) for x in Input(5)]))

In [ ]:
%timeit quantify(jumps2([int(x) for x in Input(5)]))

## [Day 6](http://adventofcode.com/2017/day/6): Memory Reallocation
Determine how many cycles it takes before a previously seen permutation of the banks input is seen again.

Let's start with reading the input

In [ ]:
def banks(): return [int(x) for x in Input(6).read().split()]
banks()

The task consists of three parts: find the bank with most blocks, redistribute those blocks, count how many cycles before we're encountering a permutation of banks that has been seen before. I represent each of these tasks with a separate function.

The instructions for restributing the blocks involves a special case for resetting the source bank to zero and fiddly index manipulations. I would have liked to write that part in a closed form list comprehension but it looks clearer this way.

The redistribute part is easier to do inplace with a list than a tuple, but the set requires a tuple as key. Can that be simplified?

In [ ]:
def most_blocks(banks):
    b = max(banks)
    return banks.index(b), b

def redistribute(banks):
    N = len(banks)
    pos, val = most_blocks(banks)
    banks[pos] = 0
    banks[:] = [x + val // N for x in banks]
    for x in range(pos+1, pos+1+ val % N):
        banks[x % N] += 1
    return tuple(banks)

def count_cycles(banks):
    history = {tuple(banks)}
    for cycles in count_from(1):
        result = redistribute(banks)
        if result in history:
            return cycles
        history.add(result)

count_cycles(banks())

In **part 2** we're asked to find the number of cycles from the first duplicated permutation until the next time the same permutation is generated.

I had hoped that I could just reuse the count_cycles() function from part 1, but I didn't manage to accomplish that.

In [ ]:
def count_cycles2(banks):
    history ={tuple(banks)}
    prev = None
    first_found = None
    for cycles in count_from(1):
        result = divide_between(banks)
        if result == prev:
            return cycles - first_found
        if result in history and prev ==  None:
            prev = result
            first_found = cycles
        history.add(result)
count_cycles2(banks())

## [Day 7](http://adventofcode.com/2017/day/7): Recursive Circus
TODO(dannas): Write description

In [21]:
class Node:
    def __init__(self, name="", parent=None, weight=0, children=None):
        self.name = name
        self.parent = parent
        self.weight = weight
        self.children = children if children else []
    
    def __str__(self):
        children = ', '.join(c.name for c in self.children)
        return '{} ({}) -> {}'.format(self.name, self.weight, children)
    def __repr__(self):
        return str(self)
        
def parse(line):
    names = re.findall(r'[a-z]+', line)
    weight = ints(line)[0]
    return names[0], weight, names[1:]

def build_tree(input):
    tower = defaultdict(Node)
    for name, weight, children in [parse(L.strip()) for L in input]:
        for c in children:
            tower[c].parent = tower[name]
        tower[name].name = name
        tower[name].weight = weight
        tower[name].children = [tower[c] for c in children]
    return tower

def find_root(tree):
    node = first(tree.values())
    while node.parent:
        node = node.parent
    return node

tower = build_tree(Input(7))
find_root(tower)

cyrupz (55) -> whbqia, sopjux, cfcdlh, wuluv, uwmocg, xtcpynj, qjvtm

In **part 2** we're asked to determine which single program has a weight that makes the tower unbalanced.

The tower is naturally represented as a tree; each program can have only parent but many children and they're all rooted in one single base program. 

In [23]:
example_input = """pbga (66)
xhth (57)
ebii (61)
havc (66)
ktlj (57)
fwft (72) -> ktlj, cntj, xhth
qoyq (66)
padx (45) -> pbga, havc, qoyq
tknk (41) -> ugml, padx, fwft
jptl (61)
ugml (68) -> gyxo, ebii, jptl
gyxo (61)
cntj (57)"""

tower = build_tree(Input(7))
import pprint
root = find_root(tower)

def tree_weight(node):
    weight = node.weight
    for c in node.children:
        _, w = tree_weight(c)
        weight += w
    return node.weight, weight
# qjvtm
# boropxd 12154
# cwwwj 1131 8 too much

def print_subtree(root):
    for c in root.children:
        print(c.name, '(', ' + '.join(cc.name for cc in c.children), ') = ',
                c.weight, ' + '.join(str(tree_weight(x)) for x in c.children),
             ' = ', tree_weight(c))

print_subtree(tower['boropxd'])

def find_subtree(node):
    w = [tree_weight for node.children]
    # find subtrees with mismatch in sums
    # Count how many of each sum
    

slzaeep ( edshwfr + emuysfg ) =  1023 (50, 50) + (50, 50)  =  (1023, 1123)
hiotqxu ( ohlhl + lpweyw + fgfhom ) =  877 (82, 82) + (54, 82) + (22, 82)  =  (877, 1123)
qppggd ( oxcth + enbtzt + ntidj + hmibku ) =  171 (74, 238) + (62, 238) + (214, 238) + (44, 238)  =  (171, 1123)
iahug ( waangt + nqijykz + pnhmvd ) =  397 (46, 242) + (168, 242) + (242, 242)  =  (397, 1123)
cwwwj ( pdxgf + ugyebr + tillos ) =  201 (236, 310) + (82, 310) + (262, 310)  =  (201, 1131)
upfhsu ( jukhdvi + lilufvg + rodfe + hvigowx ) =  27 (116, 274) + (192, 274) + (116, 274) + (97, 274)  =  (27, 1123)
jjlodie ( jtxvw + cjwnq + mtpbt ) =  46 (80, 359) + (191, 359) + (152, 359)  =  (46, 1123)


## [Day 8](http://adventofcode.com/2017/day/8): I Heard You Like Registers
Given a list of unusual register instructions, what is the largest value in any register after completing the instructions?

Each line contains a statement of two expressions. Let's start with parsing the lines. Note that array() returns ints or floats for numbers, not strings.

In [10]:
program = array(Input(8).read())
program[:5]

[['v', 'inc', 523.0, 'if', 't', '==', 6.0],
 ['qen', 'dec', -450.0, 'if', 'lht', '!=', 10.0],
 ['jyg', 'dec', -378.0, 'if', 'lb', '!=', -6.0],
 ['k', 'inc', -994.0, 'if', 'z', '<', 6.0],
 ['gjr', 'inc', -698.0, 'if', 'hbq', '<', 9.0]]

I started out with creating two functions; one for evaluating the conditional expression and one for the inc/dec expression. But I realized that they shared a lot of symmetry. Let's piggy-back on Pythons eval function.

In [13]:
def eval_expr(lhs, op, rhs):
    expr = '{} {} {}'.format(lhs, op, rhs)
    return eval(expr)

eval_expr(1, '<', 2)

True

I choose to represent the registers as a defaultdict, which will return 0 for unseen keys.

In [ ]:
def run(program):
    regs = defaultdict(int)
    for dst, op, imm1, _, src, cmp, imm2 in program:
        if eval_expr(regs[src], cmp, imm2):
            regs[dst] = eval_expr(regs[dst], '+' if op == 'inc' else '-', imm1)
    return max(regs.values())

run(program)

In **part 2** we're asked to find the highest value held in any register during this process.

The highest value will be returned from eval_expr. I used a decorator for storing the largest return value.

In [15]:
def record_max(f):
    record_max.maxval = -1
    def _f(*args):
        ret = f(*args)
        record_max.maxval = max(record_max.maxval, ret)
        return ret
    return _f

@record_max
def eval_expr(lhs, op, rhs):
    expr = '{} {} {}'.format(lhs, op, rhs)
    return eval(expr)

run(program)
print(record_max.maxval)

3818.0


## [Day 9](http://adventofcode.com/2017/day/9): Stream Processing
TODO(dannas): Write description

In [34]:
def tokens(stream):
    it = iter(stream)
    for c in it:
        if c == '<':
            while c != '>':
                if c == '!':
                    next(it)
                c = next(it)
        else:
            yield c
        
def scan_stream(stream):
    score = 0
    nesting = 0
    for t in tokens2(stream):
        if t == '{':
            nesting += 1
        elif t == '}':
            score += nesting
            nesting -= 1
    return score

scan_stream(Input(9).read())
        

11347

In **part 2** we're asked to count how many non-canceled characters are within the garbage in the puzzle input.

In [32]:
def garbage(stream):
    N = 0
    it = iter(stream)
    for c in it:
        if c == '<':
            c = next(it)
            while c != '>':
                if c == '!':
                    next(it)
                else:
                    N += 1
                c = next(it)       
    return N

assert garbage("<random characters>") == 17
assert garbage("<{o\"i!a,<{i<a>") == 10
assert garbage("<<<<>") == 3
assert garbage("<{!>}>") == 2
assert garbage("<>") == 0
assert garbage("<!!>") == 0
assert garbage("<!!!>>") == 0
garbage(Input(9).read())


5404

## [Day 10](http://adventofcode.com/2017/day/10): Knot Hash
Calculate a hashsum given a list of lengths. The hash sum is calculated by first reversing spans whose lengths are determined by the input; then the sum is the result of multiplying the first two numbers in the list.

I had trouble reversing the cyclic list. Solved it by brute force - copying the part before the current position to the end, then joining the parts back together.

In [177]:
def reverse(numbers, pos, N):
    copy = numbers[:] + numbers[:pos]
    copy[pos:pos+N] = reversed(copy[pos:pos+N])
    return copy[len(numbers):] + copy[pos:len(numbers)]
def forward(pos, length, skip_size, N):
    return (pos + length + skip_size) % N
    
reverse([0, 1, 2], 0, 3) == [2, 1, 0]
reverse([0, 1, 2], 2, 2) == [2, 1, 0]
reverse([0, 1, 2], 2, 3) == [0, 2, 1]

def hash(lengths, N=256, verbose=False):
    numbers = list(range(N))
    pos = 0
    skip_size = 0
    for L in lengths:
        numbers = reverse(numbers, pos, L)
        if verbose: print(pos, L, skip_size, numbers)
        pos = forward(pos, L, skip_size, N)
        skip_size += 1
    return numbers
hash([3, 4, 1, 5], N=5) == [3, 4, 2, 1, 0]
lengths = [int(x) for x in InputStr(10).split(',')]
x, y, *_ = hash(lengths)
x * y

7888

In **part 2** we're asked to process input of bytes instead of numbers. Many steps, but each one is mechanical.

I wasted some time trying to use a list comprehension for dense_hash. 

In [216]:
def hash_round(numbers, pos, skip_size, lengths, N=256):
    for L in lengths:
        numbers = reverse(numbers, pos, L)
        pos = forward(pos, L, skip_size, N)
        skip_size += 1
    return numbers, pos, skip_size

def sparse_hash(lengths, N=256):
    numbers = list(range(N))
    pos = 0
    skip_size = 0
    for _ in range(64):
        numbers, pos, skip_size = hash_round(numbers, pos, skip_size, lengths)
    return numbers

def dense_hash(numbers):
    h = []
    for offset in range(0, 256, 16):
        x = numbers[offset]
        for j in range(offset+1, offset+16):
            x ^= numbers[j]
        h.append(x)
    return h

def hex_nibble(nibble):
    return '0123456789abcdef'[nibble]

def to_hex(val):
    return hex_nibble(val >> 4) + hex_nibble(val & 0x0f)

def knot_hash(bytes):
    lengths = [ord(x) for x in bytes]
    lengths += [17, 31, 73, 47, 23]
    numbers = sparse_hash(lengths)
    numbers = dense_hash(numbers)
    return cat([to_hex(x) for x in numbers])
    
assert knot_hash([]) == 'a2582a3a0e66e6e86e3812dcb672a272'
assert knot_hash("AoC 2017") == '33efeb34ea91902bb2f59c9920caa6cd'

knot_hash(InputStr(10))

'decdf7d377879877173b7f2fb131cf1b'

## [Day 11](http://adventofcode.com/2017/day/11): Hex Ed
TODO(dannas): Write description

In [13]:
directions = dict(s=(0, -2), sw=(-1, -1), se=(1, -1), n=(0, 2), nw=(-1, 1), ne=(1, 1))

def bfs(goal, startpos):
    visited = set()
    queue = deque()
    queue.append([startpos])
    while queue:
        steps = queue.popleft()
        for name, d in directions.items():
            newpos = move(steps[-1], d)
            if newpos not in visited:
                visited.add(newpos)
                newlist = steps + [newpos]
                if newpos == goal:
                    return newlist
                queue.append(newlist)
            
def walk(steps):
    pos = (0, 0)
    for d in steps:
        pos = move(pos, directions[d])
    return pos

#walk('ne,ne,ne'.split(','))
#walk('ne,ne,s,s'.split(','))
goal = walk(Input(11).read().strip().split(','))
print(goal)
path = bfs(goal, (0, 0))
len(path) - 1

(473, 891)


683

In **part 2** we're asked how many steps away he is as furthest from the starting position.

Now I can't rely on a brute force search. I'll have to figure out how to calculate the distance function for hex grids.

TODO(dannas): Solve part 2.

In [14]:
walk(Input(11).read().strip().split(','))

(473, 891)

In [21]:
pos = (0, 0)
print(X(pos))
while X(pos) < 473:
    pos = move(pos, directions['ne'])
print(pos)
print(891 - Y(pos))
print(473 + 418)

0
(473, 473)
418
891


In [3]:
def L(pos):
    return pos[0]
def U(pos):
    return pos[1]
def R(pos):
    return pos[2]

def move(pos, direction):
    return (L(pos) + L(direction), U(pos) + U(direction), R(pos) + R(direction))

def walk(steps):
    # left, up, right
    pos = (0, 0, 0)
    path = [pos]
    directions = dict(s=(0, -1, 0), sw=(0, -1, -1), se=(-1, -1,  0), n=(0, 1,  0), nw=(1, 1, 0), ne=(0, 1, 1))
    for d in steps:
        pos = move(pos, directions[d])
        path.append(pos)
    return path


path = walk(Input(11).read().strip().split(','))

def hexgrid_distance(p, q=(0, 0, 0)):
    diff = (L(p) - L(q), U(p) - U(q), R(p) - R(q))
    return max(mapt(abs, (diff)))

max(path, key=hexgrid_distance)


829

## [Day 12](http://adventofcode.com/2017/day/12): Digital Plumber
We are given a list of connections between "programs". Find out how many programs are reachable from the program with id 0?

This sounds like a good fit for the union-find algorithm. Start with identifying the largest id.

In [29]:
N = max(x for L in Input(12) for x in ints(line))
N

1999

Create an id array with N+1 elements. The value at each position represents the group id. So a[0] = 42 means that 0 belongs to group 42.

In [33]:
a = list(range(0, N+1))

def find(a, x):
    return a[x]

def union(a, x, y):
    px = find(a,x)
    py = find(a, y)
    if px ==  py:
        return
    for i in range(0, N+1):
        if a[i] == px:
            a[i] = py

Loop over pairs and call union on them.

In [34]:
def parse(line):
    x, neighbors = line.split('<->')
    for n in neighbors.split(', '):
        yield int(x), int(n)

for x, y in (pair for line in Input(12) for pair in parse(line)):
    union(a, x, y)

len([x for x in a if x == a[0]])

239

In **part 2** we're asked how many groups are there in total?

That would be the number of unique group ids.

In [31]:
len(set(a))

215

## [Day 13](http://adventofcode.com/2017/day/13): Packet Scanners

Return the severity, defined as the sum of layer*depth, of getting through a firewall.

In [35]:
def is_caught(num_steps, layer_range):
    if layer_range <= 2:
        return (num_steps % layer_range) == 0
    return num_steps % (layer_range + layer_range - 2) == 0

def caught_at(layers):
    return (kv for kv in layers if is_caught(*kv))

layers = [ints(L) for L in Input(13)]    
sum(layer * depth for layer, depth in caught_at(layers))

1876

In **part 2** we're tasked with finding the fewest number of picoseconds that we need to delay the packets to pass through the firewall without getting caught.

I was mislead by the long time taken for this calculation and aborted it prematurely.

In [34]:
def find_delay(layers):
    for delay in count_from(0):
        path = [(k+delay, v) for k, v in layers]
        if quantify(kv for kv in caught_at(path)) == 0:
            return delay
        
find_delay(layers)

3964778

## [Day 14](http://adventofcode.com/2017/day/14): Disk defragmentation
Reverse engineer a hash to determine how many squares in a grid are used.

In [231]:
def row_key(key_string, row):
    return '{}-{}'.format(key_string, row)

def bits_count(x):
    return bin(x).count('1')

def hamming_weight(hash_digits):
    return sum(bits_count(x) for x in hash_digits)
    
def hash_ints(hash):
    return [int(hash[i:i+2], base=16) for i in range(0, 32, 2)]

def squares_used(key_string):
    n = 0
    for row in range(128):
        key = row_key(key_string, row)
        hash = knot_hash(key)
        numbers = hash_ints(hash)
        n += hamming_weight(numbers)
    return n    
        
key_string =  'hxtvlmkl'

squares_used(key_string)

8214

In **part 2** we're asked to count the number of regions. A region consists of a group of used squares that are adjacent, not including diagonals.

I decided to use a breadth first search and color the visited squares with a serial number. To avoid messy border calculations I inserted a border of '.' sentinels.

Had several bugs.
* off by one errors for the range of points
* to_binstring did not include leading zeroes
* to_binstring had an off-by-one in size argument to str.format()
* I used x,y for grid[X(P)][Y(P)]

The root cause of these problems was that I hadn't split the problem up in a way that allowed me to test the parts separately.


In [281]:
def to_binstring(hexstring):
    return '{:0128b}'.format(int(hexstring, base=16))

def create_grid(key_string):
    rows = []
    for row in range(128):
        key = row_key(key_string, row)
        hash = knot_hash(key)
        row = to_binstring(hash)
        row = '.' + row + '.'
        rows.append(row)
    rows.insert(0, '.' * 130)
    rows.append('.' * 130)
    return rows

def find_neighbors(grid):
    points = ((x, y) for x in range(1, 129) for y in range(1, 129))
    visited = {}
    color = 1
    for P in points:
        if grid[X(P)][Y(P)] == '1' and P not in visited:
            queue = deque()
            queue.append(P)
            while len(queue) > 0:
                pos = queue.popleft()
                visited[pos] = color
                for n in neighbors4(pos):
                    assert X(n) < len(grid)
                    assert X(n) >= 0
                    assert Y(n) < len(grid[0])
                    assert Y(n) >= 0
                    if grid[X(n)][Y(n)] == '1' and n not in visited:
                        queue.append(n)
            color += 1
    return max(visited.values())

grid = create_grid(key_string)

find_neighbors(grid)


1093

## [Day 15](http://adventofcode.com/2017/day/15): Dueling Generators
Count the number of times the lower 16 bits of numbers from two sequences matches.

I had 0xff instead of 0xffff as bitmask. Took me a while to spot.

In [301]:
def gen_a(seed):
    x = seed
    while True:
        x = (x * 16807) % 2147483647
        yield x
def gen_b(seed):
    x = seed
    while True:
        x = (x * 48271) % 2147483647
        yield x
    
MILLION = 10**6
def count_score(a_seed, b_seed, N = 40 * MILLION):
    score = 0
    for a, b in zip(gen_a(a_seed), gen_b(b_seed)):
        if (a & 0xffff) == (b & 0xffff):
            score += 1
        N -= 1
        if N == 0:
            return score
count_score(722, 354)


612

In **part 2** we're asked to only generate numbers that are multiples of 4 respectively 8.

In [303]:
def gen_a(seed):
    x = seed
    while True:
        x = (x * 16807) % 2147483647
        if x % 4 == 0:
            yield x
def gen_b(seed):
    x = seed
    while True:
        x = (x * 48271) % 2147483647
        if x % 8 == 0:
            yield x
    
MILLION = 10**6
def count_score(a_seed, b_seed, N = 5 * MILLION):
    score = 0
    for a, b in zip(gen_a(a_seed), gen_b(b_seed)):
        if (a & 0xffff) == (b & 0xffff):
            score += 1
        N -= 1
        if N == 0:
            return score
#count_score(65, 8921)
count_score(722, 354)

285

## [Day 16](http://adventofcode.com/2017/day/16): Permutation Promenade
TODO(dannas): Write description

In [27]:
def dance(programs, moves):
    
    def spin(n):
        programs[:] = programs[-n:] + programs[:-n]
    def exchange(p, q):
        t = programs[p]
        programs[p] = programs[q]
        programs[q] = t
    def partner(x, y):
        p = programs.index(x)
        q = programs.index(y)
        exchange(p, q)
    for move in moves:
        if move[0] == 's':
            spin(*ints(move))
        elif move[0] == 'x':
            exchange(*ints(move))
        else:
            partner(move[1], move[3])
    return cat(programs)

assert dance(list('abcde'), 's1 x3/4 pe/b'.split()) == 'baedc'

programs = list('abcdefghijklmnop')
moves = Input(16).read().split(',')

dance(programs, moves)


'gkmndaholjbfcepi'

In **part 2** we're asked to do the dance one billion times.

Measuring the execution time for dance, gives a runtime on my machine of ~20ms. That adds up to almost a year of runtime.

In [24]:
def memo(f):
    cache = {}
    def _f(*args):
        try:
            return cache[args]
        except KeyError:
            cache[args] = result = f(*args)
        return result
    return _f     

def dance2(programs, moves):
    
    def spin(n):
        programs[:] = programs[-n:] + programs[:-n]
    def exchange(p, q):
        t = programs[p]
        programs[p] = programs[q]
        programs[q] = t
    def partner(x, y):
        p = programs.index(x)
        q = programs.index(y)
        exchange(p, q)
    programs = list(programs)
    for move in moves:
        if move[0] == 's':
            spin(*move[1:])
        elif move[0] == 'x':
            exchange(*move[1:])
        else:
            partner(*move[1:])
    return tuple(programs)

def parse(move):
    if move[0] == 's':
        return ('s', *ints(move))
    elif move[0] == 'x':
        return ('x', *ints(move))
    else:
        return ('p', move[1],  move[3])
    
programs = tuple('abcdefghijklmnop')
moves = [parse(move) for move in Input(16).read().split(',')]

def keep_on_dancing(programs, moves, N=5000):
    cache = {}
    r = programs
    for _ in range(N):
        indata = r
        try:
            r = cache[indata]
        except KeyError:
            r = cache[indata] = dance2(indata, moves)
    return cat(r)
        
keep_on_dancing(programs, moves, N=10**9)


'abihnfkojcmegldp'

## [Day 17](http://adventofcode.com/2017/day/17): Spinlock


In [38]:
def spin(until, num_steps):
    L = [0]
    pos = 0
    for i in range(1, until+1):
        pos = (pos + num_steps) % len(L)
        L.insert(pos+1, i)
        pos = (pos + 1) % len(L)
        #print(L)
    print(L[pos-5:pos+5])
    print(L[pos])
    return L[pos+1]

[0, 1, 2, 42]

In [83]:
class Node:
    def __init__(self, val=None, next=None):
        self.val = val
        self.next = next if next else self
    def __str__(self):
        return str(self.val)
    def __repr__(self):
        return str(self)

def insert(prev, val):
    node = Node(val, prev.next)
    prev.next = node
    return node

def print_nodes(root, current):
    node = root
    s = ""
    while True:
        if id(node) == id(current):
            s += "(" + str(node.val) + ") "
        else:
            s += str(node.val) + " "
        node = node.next
        if id(node) == id(root):
            break
    print(s)

def spin3(until, num_steps):
    root = Node(0)
    node = root
    for i in range(1, until+1):
        for _ in range(num_steps):
            node = node.next
        node = insert(node, i)
        print_nodes(root, node)
    return node.next.val
    

spin3(10, 638)
        


0 (1) 
0 (2) 1 
0 2 (3) 1 
0 2 (4) 3 1 
0 2 (5) 4 3 1 
0 2 5 4 3 1 (6) 
0 2 (7) 5 4 3 1 6 
0 2 (8) 7 5 4 3 1 6 
0 2 8 (9) 7 5 4 3 1 6 
0 2 8 (10) 9 7 5 4 3 1 6 


9

## [Day 18](http://adventofcode.com/2017/day/18): Duet
TODO(dannas): Write description

In [87]:
InputStr(18).split('\n')[:10]

['set i 31',
 'set a 1',
 'mul p 17',
 'jgz p p',
 'mul a 2',
 'add i -1',
 'jgz i -2',
 'add a -1',
 'set i 127',
 'set p 316']

In [123]:
def run(instructions, verbose=False):
    regs = defaultdict(int)
    pc = 0
    speaker = 0
    def lookup(operand):
        return operand if isinstance(operand, int) else regs[operand]
    while pc >= 0 and pc < len(instructions):
        op, arg1, *arg2 = instructions[pc]
        if   op == 'snd': speaker = lookup(arg1)
        elif op == 'set': regs[arg1]  = lookup(*arg2)
        elif op == 'add': regs[arg1] += lookup(*arg2)
        elif op == 'mul': regs[arg1] *= lookup(*arg2)
        elif op == 'mod': regs[arg1] %= lookup(*arg2)
        elif op == 'rcv': 
            if lookup(arg1) != 0:
                return speaker
        elif op == 'jgz':
            if lookup(arg1) > 0:
                pc += lookup(*arg2) - 1
        pc += 1
        if verbose: print(pc, op, arg1, *arg2, regs.items(), speaker)
    
    return regs

instructions = """
set a 1
add a 2
mul a a
mod a 5
snd a
set a 0
rcv a
jgz a -1
set a 1
jgz a -2
""".strip()

run(array(instructions), verbose=True) == 4

1 set a 1 dict_items([('a', 1)]) 0
2 add a 2 dict_items([('a', 3)]) 0
3 mul a a dict_items([('a', 9)]) 0
4 mod a 5 dict_items([('a', 4)]) 0
5 snd a dict_items([('a', 4)]) 4
6 set a 0 dict_items([('a', 0)]) 4
7 rcv a dict_items([('a', 0)]) 4
8 jgz a -1 dict_items([('a', 0)]) 4
9 set a 1 dict_items([('a', 1)]) 4
7 jgz a -2 dict_items([('a', 1)]) 4
6 jgz a -1 dict_items([('a', 1)]) 4


True

In [124]:
run(array(InputStr(18)))

2951

In [125]:
class Program:
    def __init__(self, pid):
        self.pid = pid                
    def run(self, instructions, verbose=False):
        regs = defaultdict(int)
        regs['p'] = self.pid
        pc = 0
        def lookup(operand):
            return operand if isinstance(operand, int) else regs[operand]
        while pc >= 0 and pc < len(instructions):
            op, arg1, *arg2 = instructions[pc]
            if   op == 'snd': speaker = lookup(arg1)
            elif op == 'set': regs[arg1]  = lookup(*arg2)
            elif op == 'add': regs[arg1] += lookup(*arg2)
            elif op == 'mul': regs[arg1] *= lookup(*arg2)
            elif op == 'mod': regs[arg1] %= lookup(*arg2)
            elif op == 'rcv': 
                if lookup(arg1) != 0:
                    return speaker
            elif op == 'jgz':
                if lookup(arg1) > 0:
                    pc += lookup(*arg2) - 1
            pc += 1
            if verbose: print(pc, op, arg1, *arg2, regs.items(), speaker)

In [126]:
P = Program(0)
P.run(array(InputStr(18)))

2951

## [Day 19](http://adventofcode.com/2017/day/19): A Series of Tubes
Follow an ASCII diagram chart, recording the letters that you we pass. 

I used a border around the grid, for simplifying the neighbor checks. I forgot to add the lower and upper border in create_grid. Took some time before I figured that out.

I was under the impression that the grid tube didn't have to have a '+' when it changed direction. That would have simplified the find_direction function.

In **part 2** we're asked how many steps was needed.  That's just a matter of counting number of iterations of the loop in walk().

In [339]:
DOT = '.'
E, N, W, S = [(1, 0), (0, 1), (-1, 0), (0, -1)]
directions = set([E, N, W, S])
UNKNOWN = (0, 0)

def opposite_direction(direction):
    if direction == E:
        return W
    if direction == W:
        return E
    if direction == N:
        return S
    if direction == S:
        return N

def create_grid(): 
    grid = [DOT + L.rstrip('\n') + DOT for L in Input(19)]
    empty = [DOT] * len(grid[0])
    return [empty] + grid + [empty]

def move(pos, direction):
    return (X(pos) + X(direction), Y(pos) + Y(direction))

def find_start(grid):
    pos = grid[1].find('|')
    return (1, pos), S

def find_direction(grid, pos, direction):
    c = grid[X(pos)][Y(pos)]
    # Can we just continue in the same direction?
    newpos = move(pos, direction)
    if grid[X(newpos)][Y(newpos)] in '|-+' or grid[X(newpos)][Y(newpos)].isalpha():
        return pos, direction
    # What direction should we choose next?
    for d in directions - set([opposite_direction(direction)]):
        newpos = move(pos, d)
        if grid[X(newpos)][Y(newpos)] in '|-':
            return pos, d
    return pos, UNKNOWN

grid = create_grid()
pos, direction = find_start(grid)

def walk(grid):
    letters = ''
    pos, direction = find_start(grid)
    num_steps = 0
    while direction != UNKNOWN:
        c = grid[X(pos)][Y(pos)]
        if c.isalpha():
            letters += c
        pos, direction = find_direction(grid, pos, direction)
        pos = move(pos, direction)
        num_steps += 1
    return letters, num_steps
        
walk(grid)


('LXWCKGRAOY', 17302)

## [Day 20](http://adventofcode.com/2017/day/20): Particle Swarm
We're given position, velocity and acceleration for a set of particles. Each tick, all particles are updated simultaenously. Which particle will stay closest to position <0, 0, 0> in the long term?

The calculation of positions and velocity is opaque. I should use better names or extract code into functions.

In [362]:
def coordinates(string):
    matches = re.findall(r'-?\d+', string)
    return mapt(int, matches)

def add3(v1, v2):
    return (v1[0] + v2[0], v1[1] + v2[1], v1[2] + v2[2])

def cityblock_distance3(P, q=(0, 0, 0)): 
    "City block distance between two points in 3d space."
    p = P[0]
    return abs(p[0] - q[0]) + abs(p[1] - q[1]) + abs(p[2] - q[2])

particles = [mapt(coordinates, line.split(', ')) for line in Input(20)]

for _ in range(1000):
    particles = [(P[0], add3(P[1], P[2]), P[2]) for P in particles]
    particles = [(add3(P[0], P[1]), P[1], P[2]) for P in particles]

val = min(particles, key=cityblock_distance3)
particles.index(val)

119

In **part 2** we're asked to count how many particles are left if we filter out collisions.

In [366]:
particles = [mapt(coordinates, line.split(', ')) for line in Input(20)]

for _ in range(1000):
    particles = [(P[0], add3(P[1], P[2]), P[2]) for P in particles]
    particles = [(add3(P[0], P[1]), P[1], P[2]) for P in particles]
    counts = Counter([P[0] for P in particles])
    particles = [P for P in particles if counts[P[0]] == 1]
len(particles)

471

## [Day 21](http://adventofcode.com/2017/day/21): Fractal Art


In [370]:
image = """
.#.
..#
###""".strip().split()
image

def size(image):
    return len(image)

def rotates_and_flips(image):
    # Rotate on x?
    # Rotate on y?
    # Rotate on both?
    # Flip one line?
    # Flip multiple lines?
    

image

['.#.', '..#', '###']

## [Day 22](http://adventofcode.com/2017/day/22): Sporifica Virus
Given a map of "computer nodes" that may be infected by a virus or not, determine how many burst of activity cause a node to become infected. The map only contains the center of the grid; there are many more nodes beyond those shown, but none of them are currently infected.

Looks like a graph walk problem, where the bounds of the graph is not known. I'll represent the graph as a list of strings

In [39]:
DIRECTIONS = W, N, E, S = [(-1, 0), (0, -1), (1, 0), (0, 1)]
STR_DIRECTIONS = {W:'W', N:'N', E:'E', S:'S'}

def create_grid(string):
    lines = string.splitlines()
    grid = defaultdict(lambda: str('.'))
    N = len(lines)
    startpos = N//2,  N//2
    for y, row in enumerate(lines):
        for x, node in enumerate(row):
            grid[(x, y)] = node
    return grid, startpos
          
def turn_left(direction):
    index = DIRECTIONS.index(direction)
    return DIRECTIONS[index-1]

def turn_right(direction):
    index = DIRECTIONS.index(direction)
    return DIRECTIONS[index-3]

def reverse(direction):
    index = DIRECTIONS.index(direction)
    return DIRECTIONS[index-2]

num_infecting_bursts = 0
def burst(pos, direction, grid):
    global num_infecting_bursts
    if grid[pos] == '#':
        grid[pos] =  '.'
        direction = turn_right(direction)
    else:
        grid[pos] = '#'
        direction = turn_left(direction)
        num_infecting_bursts += 1
    pos = move(pos, direction)
    return pos, direction, grid

grid, pos = create_grid(InputStr(22))
direction = N
for _ in range(10000):
    pos, direction, grid = burst(pos, direction, grid)
print(num_infecting_bursts)


5369


In [38]:
num_infecting_bursts = 0
def burst2(pos, direction, grid):
    global num_infecting_bursts
    node = grid[pos]
    if node ==  '.':
        grid[pos] = 'W'
        direction = turn_left(direction)
    elif node == 'W':
        grid[pos] = '#'
        num_infecting_bursts += 1
    elif node == '#':
        grid[pos] = 'F'
        direction = turn_right(direction)
    elif node == 'F':
        grid[pos] = '.'
        direction = reverse(direction)
    pos = move(pos, direction)
    return pos,  direction, grid

grid, pos = create_grid(InputStr(22))
direction = N
for _ in range(10000000):
    pos, direction, grid = burst2(pos, direction, grid)
print(num_infecting_bursts)

2510774


## [Day 23](adventofcode.com/2017/day/23): Coprocessor Conflagration


In [2]:
def run(instructions, debug=False):
    regs = defaultdict(int)
    if debug:
        regs['a'] = 1
    pc = 0
    num_multiplies = 0
    def lookup(operand):
        return operand if isinstance(operand, int) else regs[operand]
    while pc >= 0 and pc < len(instructions):
        op, arg1, *arg2 = instructions[pc]
        if   op == 'set': regs[arg1]  = lookup(*arg2)
        elif op == 'sub': regs[arg1] -= lookup(*arg2)
        elif op == 'mul': 
            regs[arg1] *= lookup(*arg2)
            num_multiplies += 1
        elif op == 'jnz':
            if lookup(arg1) != 0:
                pc += lookup(*arg2) - 1
        pc += 1
    
    return num_multiplies, regs['h']

run(array(InputStr(23)))


(6241, 1)

In **part 2** we're asked to determine what value the h-register would have if the computation were to terminate.

I let the program run for a few minutes, and sure enough it didn't finish. How go about this one? The only calculations that this program needs to do, are the ones that ultimately have effect on 'h'.

I seem to recall that compiler people talk about data dependencies and control flow dependencies. Are those two orthogonal axis that you explore when you do optimizations?
I know of strenght reductions and constant folding, but those won't help much here.

A program can be described as consisting of blocks that are separated by control flow. Can any of such basic blocks be eliminated. If so, how?

How can I show the basic block of the program?
Maybe start with illustrating jnz src and dst?

In [3]:
instructions = array(InputStr(23))

jumps = [(src, src+y-1) for src, (op, x,  y) in enumerate(instructions) if op == 'jnz']

def diff(r):
    x, y = r
    return x-y
jumps.sort(key=diff, reverse=True)
print(jumps)
N = len(instructions)
M = len(jumps)
prefix = [' '] * M
margin = []
for _ in range(N):
    margin.append(prefix[:])

for col, (src, dst) in enumerate(jumps):
    margin[src][col] = '+'
    margin[dst][col] = '^'
    if src > dst:
        src, dst = dst, src
    for i in range(src+1, dst):
        margin[i][col] = '|'
margin = [''.join(row) for row in margin]
for i in range(N):
    print(margin[i], instructions[i])

[(31, 7), (23, 9), (19, 10), (2, 3), (14, 15), (24, 25), (28, 29), (29, 31), (3, 7)]
          ['set', 'b', 81]
          ['set', 'c', 'b']
   +      ['jnz', 'a', 2]
   ^    + ['jnz', 1, 5]
        | ['mul', 'b', 100]
        | ['sub', 'b', -100000]
        | ['set', 'c', 'b']
^       ^ ['sub', 'c', -17000]
|         ['set', 'f', 1]
|^        ['set', 'd', 2]
||^       ['set', 'e', 2]
|||       ['set', 'g', 'd']
|||       ['mul', 'g', 'e']
|||       ['sub', 'g', 'b']
||| +     ['jnz', 'g', 2]
||| ^     ['set', 'f', 0]
|||       ['sub', 'e', -1]
|||       ['set', 'g', 'e']
|||       ['sub', 'g', 'b']
||+       ['jnz', 'g', -8]
||        ['sub', 'd', -1]
||        ['set', 'g', 'd']
||        ['sub', 'g', 'b']
|+        ['jnz', 'g', -13]
|    +    ['jnz', 'f', 2]
|    ^    ['sub', 'h', -1]
|         ['set', 'g', 'b']
|         ['sub', 'g', 'c']
|     +   ['jnz', 'g', 2]
|     ^+  ['jnz', 1, 3]
|      |  ['sub', 'b', -17]
+      ^  ['jnz', 1, -23]


In [8]:
def run_b():
    a = b = c = d = e = f = g = h = 0
    b = 81
    c = b
    # jnz a 2
    # jnz 1 5
    if a:
        b *= 100
        b -= -100000
        c = b
    while True:
        c -= -17000
        f = 1
        while True:
            d = 2
            while True:
                e = 2
                g = d
                g *= e
                g -= b
                if g:
                    # jnz g 2
                    f = 0
                e -= -1
                g = e
                g = b
                if g:
                    break
                # jnz g -8
            d -= -1
            g = d
            g -= b
            if g:
                break
            # jnz g -13
        if f:
            # jnz f 2
            h -= -1
        g = b
        g -= c
        if g:
            return h
        # jnz g 2
        # jnz 1 3
        b -= -17
        # jnz 1 -23
run_b()

0

## [Day 24](http://adventofcode.com/2017/day/24): Electromagnetic Moat
Build the strongest bridge possible from magnetic components. The strength of a bridge is the sum of the port types in each component.

Let's start by parsing the components:

In [29]:
components = set(ints(line) for line in Input(24))
len(components)


57

We need to find the maximum bridge. Sounds like a search problem. Should we use A* search or is the number of possibilities small enough to allow an exhaustive depth first search?

In [4]:
# Use a tree
# Use a graph
# How keep track of which components has been selected
# Store remaining items together with selected items?

In [36]:
def find_matches(number, components):
    return [c for c in components if number in c]

def build_bridges(components):
    bridges = []
    queue = deque()
    for c in find_matches(0, components):
        queue.append([[c], components-{c}])
    while queue:
        used, remaining = queue.popleft()
        number = used[-1][1]
        matches = find_matches(number, remaining)
        if not matches:
            bridges.append(used)
        else:
            for c in matches:
                nc = c if c[0] == number else (c[1], c[0])
                queue.append([used + [nc], remaining - {c}])
    return bridges

test_components = [
    (0, 2), (2, 2), (2, 3), (3, 4), (3, 5), (0, 1), (10, 1), (9, 10)
]

bridges = build_bridges(components)
#bridges = build_bridges(set(test_components))

def strength(bridge):
    return sum(x+y for x, y in bridge)

max(strength(b) for b in bridges)

2006

In [41]:
sorted(((len(b), strength(b)) for b in bridges), reverse=True)

[(34, 1994),
 (34, 1994),
 (34, 1994),
 (34, 1994),
 (34, 1994),
 (34, 1994),
 (34, 1994),
 (34, 1994),
 (34, 1994),
 (34, 1994),
 (34, 1994),
 (34, 1994),
 (34, 1994),
 (34, 1994),
 (34, 1994),
 (34, 1994),
 (34, 1994),
 (34, 1994),
 (34, 1994),
 (34, 1994),
 (34, 1994),
 (34, 1994),
 (34, 1994),
 (34, 1994),
 (34, 1994),
 (34, 1994),
 (34, 1994),
 (34, 1994),
 (34, 1994),
 (34, 1994),
 (34, 1994),
 (34, 1994),
 (34, 1994),
 (34, 1994),
 (34, 1994),
 (34, 1994),
 (34, 1994),
 (34, 1994),
 (34, 1994),
 (34, 1994),
 (34, 1994),
 (34, 1994),
 (34, 1994),
 (34, 1994),
 (34, 1994),
 (34, 1994),
 (34, 1994),
 (34, 1994),
 (34, 1994),
 (34, 1994),
 (34, 1994),
 (34, 1994),
 (34, 1994),
 (34, 1994),
 (34, 1994),
 (34, 1994),
 (34, 1994),
 (34, 1994),
 (34, 1994),
 (34, 1994),
 (34, 1994),
 (34, 1994),
 (34, 1994),
 (34, 1994),
 (34, 1994),
 (34, 1994),
 (34, 1994),
 (34, 1994),
 (34, 1994),
 (34, 1994),
 (34, 1994),
 (34, 1994),
 (34, 1994),
 (34, 1994),
 (34, 1994),
 (34, 1994),
 (34, 1994),